In [ ]:
#import methods and modules
# (how to make plotly graph into image) https://plotly.com/python/static-image-export/
# (how to make a pdf) https://www.youtube.com/watch?app=desktop&v=Fxt2T37km9o&ab_channel=ShaneLee
# (tutorial on reportlab) https://www.youtube.com/watch?v=ZDR7-iSuwkQ&ab_channel=AllTech
# (plotly stacking bar chart) https://coderzcolumn.com/tutorials/data-science/build-dashboard-using-streamlit-and-cufflinks



import csv
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import os

from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER, TA_LEFT, TA_RIGHT
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
 
with open("/home/roycevaughn/Documents/AG_model/m3.csv") as f:
    raw_data = dict(zip(*csv.reader(f)))

file_path = 'output123.txt'
output_file = open(file_path, 'w')

# logan standard test measures: Sample ID, TEC, pH, Organic_matter_percent, Exchangeable Hydrogen, Ca, Mg, K, Na, P, S, B, Fe, Mn, Cu, Zn, Al

# gather data from test and user
test_id = "standard_test" # Agger Grand Test Name indicates which variables will be used in the model

# entering in data from the soil test
CompanyID = 13266
CompanyName = "Agger Grand Company"
JobName = "Royce Vaughn"
RunDate = "09/01/2022"
BatchDate = "08/29/2022" 
SubmittedBy = "Royce Vaughn"
BatchID = 152253
SampleLocation = "Exeter"
LabNumber = 40  
SampleDepth = 6
TEC_meq = 7.6
pH = 7.1
OrganicMatter_percent = 10.19
S_ppm = 9
# if P is in P2O5 then divide by 2.29 to get elemental P. The figure will be noted as such on the soil test.
P_ppm = 143.45
Ca_ppm = 1275
Mg_ppm = 79.5
K_ppm = 54
Na_ppm = 22
Exchangeable_H_base_saturation = 0
Other_bases_base_saturation = 4.3
B_ppm = 0.44
Fe_ppm = 179
Mn_ppm = 21
Cu_ppm = 18.13
Zn_ppm = 125.2
Al_ppm = 567
CN_ratio = 16.48
Co_ppm = 0.098
Mo_ppm = 0
Cl_ppm = 0
Ammonium_ppm = 0.3
Nitrogen_ppm = 1.5
Nitrate_ppm = 3.6
Se_ppm = 0.17
Si_ppm = 34.3
ENR_lbacre = 125
EC_mmhoscm = 0
bulk_density = 1.33 # in units of grams per cubic cm. Usually 1.33 for soil. assume 1.33 if no figure is on soil test

In [18]:
# CUSTOMER SUPPLIED INFORMATION
soil_type = "dirt" #default setting is dirt.
soil_type = "dirt" #dirt, compost, raised bed with "garden soil"

cultivation_type = "no-till" # no-till, normal-till (to 7 inches), deep-till (over 7 inches)
cultivation_depth = 6.75 #default setting is 6.75 inches
if cultivation_type == "no-till":
    cultivation_depth = 4
if cultivation_type == "normal-till":
    cultivation_depth = 6.75
if cultivation_depth == "deep-till":
    cultivation_depth = 10

growing_area_units = "acres" #default set to acres
growing_area_units = "square feet" #customer selects: acres, square feet, etc

unit = 43560 #this is a default for 1 acre.
unit = 1492.63 # user supplies this value, otherwise assume default.

growing_area_size_sqft = 43560 # 1 acre = 43560 sq ft. this is the default setting.
if growing_area_units == "acres":
    growing_area_size_sqft = unit * 43560
if growing_area_units == "square feet":
    growing_area_size_sqft = unit

# Calculating the weight of the growing area
weight_growing_area = cultivation_depth * growing_area_size_sqft * bulk_density * 5.2023 # in lbs

# CUSTOMER SUPPLIED IDEAL CATION BASE SATURATION %
Ca_desired_sat = 68
Mg_desired_sat = 12
K_desired_sat = 8
Na_desired_sat = 0.5
H_desired_sat = 10


In [19]:
# calculating ideal values

#finding key ratios
P_to_Zn = float(round(P_ppm/Zn_ppm, 2)) # ideal is 10:1
Ca_to_B = float(round(Ca_ppm/B_ppm, 2)) # ideal is 1000:1
P_to_K = float(round(P_ppm/K_ppm, 2)) # ideal is 1:1
K_to_S = float(round(K_ppm/S_ppm, 2)) # ideal is 2:1
K_to_Fe = float(round(K_ppm/Fe_ppm, 2)) # ideal is 4:1 (actually 4.15:1 but 4:1 looks better)
Fe_to_Mn = float(round(Fe_ppm/Mn_ppm, 2)) # ideal is 4:1 (actually 4.15:1 but 4:1 looks better)
Zn_to_Cu = float(round(Zn_ppm/Cu_ppm, 2)) # ideal is 2:1

# Setting max and min element concentrations, in ppm (taken from astera). any soil below 7 TEC these are the ideals

min_S_ppm = 50
min_P_ppm = 100
min_Ca_ppm = 1000
min_Mg_ppm = 100
min_K_ppm = 100
min_Na_ppm = 25
min_B_ppm = 1
min_Fe_ppm = 50
min_Mn_ppm = 25
min_Cu_ppm = 5
min_Zn_ppm = 10

max_S_ppm = 300
max_P_ppm = 1000
max_Zn_ppm = 50
max_Cu_ppm = 25

# calculating missing figures from soil test
H_ppm = float(round(TEC_meq * 10 * Exchangeable_H_base_saturation/100, 2))
ppm_salt = EC_mmhoscm * 640 # a unit of mmhos/cm is equal to 640 ppm salt.
N_ppm = OrganicMatter_percent * 10 #assume each 1% OM gives 10 ppm, or 20 lb per acre.

# CALCULATING BASE SATURATION

Ca_sat = float(round(Ca_ppm / (TEC_meq*10*(39.76623274 / 2)) * 100, 2))
Mg_sat = float(round(Mg_ppm / (TEC_meq*10*(24.1159311 / 2)) * 100, 2))
K_sat = float(round(K_ppm / (TEC_meq*10*(38.79415383 / 1)) * 100, 2))
Na_sat = float(round(Na_ppm / (TEC_meq*10*(22.8109313 / 1)) * 100, 2))
H_sat = float(round(H_ppm / (TEC_meq*10*(1 / 1)) * 100, 2))
Other_sat = float(round((100 - (Ca_sat + Mg_sat + K_sat + Na_sat + H_sat)), 2))
TotalBasesSaturation = Ca_sat + Mg_sat + K_sat + Na_sat + H_sat + Other_sat


#CALCULATING IDEAL PPM MAJOR CATIONS

Ca_ideal_ppm = float(round((TEC_meq*10*(39.76623274 / 2)*(Ca_desired_sat/100)), 2))
Mg_ideal_ppm = float(round((TEC_meq*10*(24.1159311 / 2)*(Mg_desired_sat/100)), 2))
K_ideal_ppm = float(round((TEC_meq*10*(38.79415383 / 1)*(K_desired_sat/100)), 2))
Na_ideal_ppm = float(round((TEC_meq*10*(22.8109313 / 1)*(Na_desired_sat/100)), 2))
H_ideal_ppm = float(round((TEC_meq*10*(1 / 1)*(H_desired_sat/100)), 2))

if TEC_meq < 7:
    Ca_ideal_ppm = float(round(min_Ca_ppm, 2))
    Mg_ideal_ppm = float(round(min_Mg_ppm, 2))
    K_ideal_ppm = float(round(min_K_ppm, 2))
    Na_ideal_ppm = float(round(min_Na_ppm, 2))
    
'''
working with lists. this is not in the model, just trying it. you can delete it
mc_ppm_ideal = [Ca_ideal_ppm, Mg_ideal_ppm, K_ideal_ppm, Na_ideal_ppm, H_ideal_ppm]
mc_ppm_actual = [Ca_ppm, Mg_ppm, K_ppm, Na_ppm, H_ppm]
mc_ppm_percent_ideal = [(x*100)/y for x,y in zip(mc_ppm_actual, mc_ppm_ideal)]
'''

#CALCULATING IDEAL PPM MAJOR ANIONS
P_ideal_ppm = float(round(K_ideal_ppm, 2))
S_ideal_ppm = float(round(1/2 * K_ideal_ppm, 2))

if TEC_meq < 7:
    P_ideal_ppm = float(round(min_P_ppm, 2))
    S_ideal_ppm = float(round(min_S_ppm, 2))

#CALCULATING IDEAL PPM MINOR ANIONS
B_ideal_ppm = float(round(1/1000 * Ca_ideal_ppm, 2))
Mo_ideal_ppm = 1
Se_ideal_ppm = 1
Si_ideal_ppm = 300

if TEC_meq < 7:
    B_ideal_ppm = min_B_ppm
    
#CALCULATING IDEAL PPM MINOR CATIONS
Fe_ideal_ppm = float(round(5/12 * K_ideal_ppm, 2))
Mn_ideal_ppm = float(round(5/12 * 5/12 * K_ideal_ppm, 2))
Zn_ideal_ppm = float(round(1/10 * P_ideal_ppm, 2))
Cu_ideal_ppm = float(round(1/20 * K_ideal_ppm, 2))
Co_ideal_ppm = 1

if TEC_meq < 7:
    Fe_ideal_ppm = float(round(min_Fe_ppm, 2))
    Mn_ideal_ppm = float(round(min_Mn_ppm, 2))
    Zn_ideal_ppm = float(round(min_Zn_ppm, 2))
    Cu_ideal_ppm = float(round(min_Cu_ppm, 2))

#CALCULATING DIFFERENCE FROM IDEAL PPM
Ca_diff_ppm = Ca_ideal_ppm - Ca_ppm
Mg_diff_ppm = Mg_ideal_ppm - Mg_ppm
K_diff_ppm = K_ideal_ppm - K_ppm
Na_diff_ppm = Na_ideal_ppm - Na_ppm
H_diff_ppm = H_ideal_ppm - H_ppm
B_diff_ppm = B_ideal_ppm - B_ppm
Mo_diff_ppm = Mo_ideal_ppm - Mo_ppm
Se_diff_ppm = Se_ideal_ppm - Se_ppm
Si_diff_ppm = Si_ideal_ppm - Si_ppm
Fe_diff_ppm = Fe_ideal_ppm - Fe_ppm
Mn_diff_ppm = Mn_ideal_ppm - Mn_ppm
Zn_diff_ppm = Zn_ideal_ppm - Zn_ppm
Cu_diff_ppm = Cu_ideal_ppm - Cu_ppm
Co_diff_ppm = Co_ideal_ppm - Co_ppm

#CALCULATING PERCENT OF IDEAL
Ca_percent_ideal = float(round((Ca_ppm / Ca_ideal_ppm)*100, 2))
Mg_percent_ideal = float(round((Mg_ppm / Mg_ideal_ppm)*100, 2))
K_percent_ideal = float(round((K_ppm / K_ideal_ppm)*100, 2))
Na_percent_ideal = float(round((Na_ppm / Na_ideal_ppm)*100, 2))
H_percent_ideal = float(round((H_ppm / H_ideal_ppm)*100, 2))
B_percent_ideal = float(round((B_ppm / B_ideal_ppm)*100, 2))
Mo_percent_ideal = float(round((Mo_ppm / Mo_ideal_ppm)*100, 2))
Se_percent_ideal = float(round((Se_ppm / Se_ideal_ppm)*100, 2))
Si_percent_ideal = float(round((Si_ppm / Si_ideal_ppm)*100, 2))
Fe_percent_ideal = float(round((Fe_ppm / Fe_ideal_ppm)*100, 2))
Mn_percent_ideal = float(round((Mn_ppm / Mn_ideal_ppm)*100, 2))
Zn_percent_ideal = float(round((Zn_ppm / Zn_ideal_ppm)*100, 2))
Cu_percent_ideal = float(round((Cu_ppm / Cu_ideal_ppm)*100, 2))
Co_percent_ideal = float(round((Co_ppm / Co_ideal_ppm)*100, 2))
P_percent_ideal = float(round((P_ppm / P_ideal_ppm)*100, 2))
S_percent_ideal = float(round((S_ppm / S_ideal_ppm)*100, 2))

# finding how many ppm excess cations we actually have
cation_excess = 0
if Ca_percent_ideal > 100:
     cation_excess = (Ca_ppm - Ca_ideal_ppm)
if Mg_percent_ideal > 100:
     cation_excess += (Mg_ppm - Mg_ideal_ppm)
if K_percent_ideal > 100:
     cation_excess += (K_ppm - K_ideal_ppm)
if Na_percent_ideal > 100:
     cation_excess += (Na_ppm - Na_ideal_ppm)

In [20]:
# create a bar chart for micro-nutrients
microdata = {'Element': ["Iron", "Manganese", "Copper", "Boron", "Zinc", "Molybdenum", "Cobalt", "Silicon", "Selenium"],
             'Level - 100 is ideal': [Fe_percent_ideal, Mn_percent_ideal, Cu_percent_ideal, B_percent_ideal, Zn_percent_ideal, Mo_percent_ideal, Co_percent_ideal, Si_percent_ideal, Se_percent_ideal]}
fig = px.bar(microdata, x='Element', y='Level - 100 is ideal')
fig.add_hrect(y0=0, y1=80, line_width=0, fillcolor="blue", opacity=0.25)
fig.add_hrect(y0=80, y1=120, line_width=0, fillcolor="green", opacity=0.25)
fig.add_hrect(y0=120, y1=200, line_width=0, fillcolor="red", opacity=0.25)
fig.add_shape(type="rect", line_width=0,
    xref="x domain",
    x0=0, x1=1, y0=0, y1=2200,
    fillcolor="Red", opacity=0.25,
)
fig.update_yaxes(range=[0, 200])
if not os.path.exists("/home/roycevaughn/Documents/AG_model/images"):
    os.mkdir("/home/roycevaughn/Documents/AG_model/images")
fig.write_image("/home/roycevaughn/Documents/AG_model/images/micro_bar.png")

In [21]:
#Create Gauge Charts

# Calcium Gauge Chart
fig = go.Figure(go.Indicator(
    mode = "gauge+number+delta",
    value = (Ca_percent_ideal),
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Calcium", 'font': {'size': 75}},
    delta = {'reference': 100, 'increasing': {'color': "black"}},
    gauge = {
        'axis': {'range': [None, 200], 'tickwidth': 1, 'tickcolor': "black"},
        'bar': {'color': "black"},
        'bgcolor': "black",
        'borderwidth': 2,
        'bordercolor': "black",
        'steps': [
            {'range': [0, 10], 'color': '#00ffd8'},
            {'range': [10, 20], 'color': '#06f7c0'},
            {'range': [20, 30], 'color': '#0beea8'},
            {'range': [30, 40], 'color': '#11e690'},
            {'range': [40, 50], 'color': '#17de78'},
            {'range': [50, 60], 'color': '#1cd560'},
            {'range': [60, 70], 'color': '#22cd48'},
            {'range': [70, 80], 'color': '#28c530'},
            {'range': [80, 90], 'color': '#2dbc18'},
            {'range': [90, 100], 'color': '#33b400'},
            {'range': [100, 110], 'color': '#47a200'},
            {'range': [110, 120], 'color': '#5c9000'},
            {'range': [120, 130], 'color': '#707e00'},
            {'range': [130, 140], 'color': '#856c00'},
            {'range': [140, 150], 'color': '#995a00'},
            {'range': [150, 160], 'color': '#ad4800'},
            {'range': [160, 170], 'color': '#c23600'},
            {'range': [170, 180], 'color': '#d62400'},
            {'range': [180, 190], 'color': '#eb1200'},
            {'range': [190, 200], 'color': '#ff0000'}],
        'threshold': {
            'line': {'color': "navajowhite", 'width': 4},
            'thickness': 0.75,
            'value': 100}}))

fig.update_layout(paper_bgcolor = "lavender", font = {'color': "black", 'family': "Arial"})

if not os.path.exists("/home/roycevaughn/Documents/AG_model/images"):
    os.mkdir("/home/roycevaughn/Documents/AG_model/images")
fig.write_image("/home/roycevaughn/Documents/AG_model/images/calcium_gauge.png")

# Potassium Gauge Chart
fig = go.Figure(go.Indicator(
    mode = "gauge+number+delta",
    value = (K_percent_ideal),
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Potassium", 'font': {'size': 75}},
    delta = {'reference': 100, 'increasing': {'color': "black"}},
    gauge = {
        'axis': {'range': [None, 200], 'tickwidth': 1, 'tickcolor': "black"},
        'bar': {'color': "black"},
        'bgcolor': "black",
        'borderwidth': 2,
        'bordercolor': "black",
        'steps': [
            {'range': [0, 10], 'color': '#00ffd8'},
            {'range': [10, 20], 'color': '#06f7c0'},
            {'range': [20, 30], 'color': '#0beea8'},
            {'range': [30, 40], 'color': '#11e690'},
            {'range': [40, 50], 'color': '#17de78'},
            {'range': [50, 60], 'color': '#1cd560'},
            {'range': [60, 70], 'color': '#22cd48'},
            {'range': [70, 80], 'color': '#28c530'},
            {'range': [80, 90], 'color': '#2dbc18'},
            {'range': [90, 100], 'color': '#33b400'},
            {'range': [100, 110], 'color': '#47a200'},
            {'range': [110, 120], 'color': '#5c9000'},
            {'range': [120, 130], 'color': '#707e00'},
            {'range': [130, 140], 'color': '#856c00'},
            {'range': [140, 150], 'color': '#995a00'},
            {'range': [150, 160], 'color': '#ad4800'},
            {'range': [160, 170], 'color': '#c23600'},
            {'range': [170, 180], 'color': '#d62400'},
            {'range': [180, 190], 'color': '#eb1200'},
            {'range': [190, 200], 'color': '#ff0000'}],
        'threshold': {
            'line': {'color': "navajowhite", 'width': 4},
            'thickness': 0.75,
            'value': 100}}))

fig.update_layout(paper_bgcolor = "lavender", font = {'color': "black", 'family': "Arial"})

if not os.path.exists("/home/roycevaughn/Documents/AG_model/images"):
    os.mkdir("/home/roycevaughn/Documents/AG_model/images")
fig.write_image("/home/roycevaughn/Documents/AG_model/images/potassium_gauge.png")

# Magnesium Gauge Chart
fig = go.Figure(go.Indicator(
    mode = "gauge+number+delta",
    value = (Mg_percent_ideal),
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Magnesium", 'font': {'size': 75}},
    delta = {'reference': 100, 'increasing': {'color': "black"}},
    gauge = {
        'axis': {'range': [None, 200], 'tickwidth': 1, 'tickcolor': "black"},
        'bar': {'color': "black"},
        'bgcolor': "black",
        'borderwidth': 2,
        'bordercolor': "black",
        'steps': [
            {'range': [0, 10], 'color': '#00ffd8'},
            {'range': [10, 20], 'color': '#06f7c0'},
            {'range': [20, 30], 'color': '#0beea8'},
            {'range': [30, 40], 'color': '#11e690'},
            {'range': [40, 50], 'color': '#17de78'},
            {'range': [50, 60], 'color': '#1cd560'},
            {'range': [60, 70], 'color': '#22cd48'},
            {'range': [70, 80], 'color': '#28c530'},
            {'range': [80, 90], 'color': '#2dbc18'},
            {'range': [90, 100], 'color': '#33b400'},
            {'range': [100, 110], 'color': '#47a200'},
            {'range': [110, 120], 'color': '#5c9000'},
            {'range': [120, 130], 'color': '#707e00'},
            {'range': [130, 140], 'color': '#856c00'},
            {'range': [140, 150], 'color': '#995a00'},
            {'range': [150, 160], 'color': '#ad4800'},
            {'range': [160, 170], 'color': '#c23600'},
            {'range': [170, 180], 'color': '#d62400'},
            {'range': [180, 190], 'color': '#eb1200'},
            {'range': [190, 200], 'color': '#ff0000'}],
        'threshold': {
            'line': {'color': "navajowhite", 'width': 4},
            'thickness': 0.75,
            'value': 100}}))
fig.update_layout(paper_bgcolor = "lavender", font = {'color': "black", 'family': "Arial"})

if not os.path.exists("/home/roycevaughn/Documents/AG_model/images"):
    os.mkdir("/home/roycevaughn/Documents/AG_model/images")
fig.write_image("/home/roycevaughn/Documents/AG_model/images/magnesium_gauge.png")

# Sodium Gauge Chart
fig = go.Figure(go.Indicator(
    mode = "gauge+number+delta",
    value = (Na_percent_ideal),
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Sodium", 'font': {'size': 75}},
    delta = {'reference': 100, 'increasing': {'color': "black"}},
    gauge = {
        'axis': {'range': [None, 200], 'tickwidth': 1, 'tickcolor': "black"},
        'bar': {'color': "black"},
        'bgcolor': "black",
        'borderwidth': 2,
        'bordercolor': "black",
        'steps': [
            {'range': [0, 10], 'color': '#00ffd8'},
            {'range': [10, 20], 'color': '#06f7c0'},
            {'range': [20, 30], 'color': '#0beea8'},
            {'range': [30, 40], 'color': '#11e690'},
            {'range': [40, 50], 'color': '#17de78'},
            {'range': [50, 60], 'color': '#1cd560'},
            {'range': [60, 70], 'color': '#22cd48'},
            {'range': [70, 80], 'color': '#28c530'},
            {'range': [80, 90], 'color': '#2dbc18'},
            {'range': [90, 100], 'color': '#33b400'},
            {'range': [100, 110], 'color': '#47a200'},
            {'range': [110, 120], 'color': '#5c9000'},
            {'range': [120, 130], 'color': '#707e00'},
            {'range': [130, 140], 'color': '#856c00'},
            {'range': [140, 150], 'color': '#995a00'},
            {'range': [150, 160], 'color': '#ad4800'},
            {'range': [160, 170], 'color': '#c23600'},
            {'range': [170, 180], 'color': '#d62400'},
            {'range': [180, 190], 'color': '#eb1200'},
            {'range': [190, 200], 'color': '#ff0000'}],
        'threshold': {
            'line': {'color': "navajowhite", 'width': 4},
            'thickness': 0.75,
            'value': 100}}))

fig.update_layout(paper_bgcolor = "lavender", font = {'color': "black", 'family': "Arial"})

if not os.path.exists("/home/roycevaughn/Documents/AG_model/images"):
    os.mkdir("/home/roycevaughn/Documents/AG_model/images")
fig.write_image("/home/roycevaughn/Documents/AG_model/images/sodium_gauge.png")

# Phosphorus Gauge Chart
fig = go.Figure(go.Indicator(
    mode = "gauge+number+delta",
    value = (P_percent_ideal),
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Phosphorus", 'font': {'size': 75}},
    delta = {'reference': 100, 'increasing': {'color': "black"}},
    gauge = {
        'axis': {'range': [None, 200], 'tickwidth': 1, 'tickcolor': "black"},
        'bar': {'color': "black"},
        'bgcolor': "black",
        'borderwidth': 2,
        'bordercolor': "black",
        'steps': [
            {'range': [0, 10], 'color': '#00ffd8'},
            {'range': [10, 20], 'color': '#06f7c0'},
            {'range': [20, 30], 'color': '#0beea8'},
            {'range': [30, 40], 'color': '#11e690'},
            {'range': [40, 50], 'color': '#17de78'},
            {'range': [50, 60], 'color': '#1cd560'},
            {'range': [60, 70], 'color': '#22cd48'},
            {'range': [70, 80], 'color': '#28c530'},
            {'range': [80, 90], 'color': '#2dbc18'},
            {'range': [90, 100], 'color': '#33b400'},
            {'range': [100, 110], 'color': '#47a200'},
            {'range': [110, 120], 'color': '#5c9000'},
            {'range': [120, 130], 'color': '#707e00'},
            {'range': [130, 140], 'color': '#856c00'},
            {'range': [140, 150], 'color': '#995a00'},
            {'range': [150, 160], 'color': '#ad4800'},
            {'range': [160, 170], 'color': '#c23600'},
            {'range': [170, 180], 'color': '#d62400'},
            {'range': [180, 190], 'color': '#eb1200'},
            {'range': [190, 200], 'color': '#ff0000'}],
        'threshold': {
            'line': {'color': "navajowhite", 'width': 4},
            'thickness': 0.75,
            'value': 100}}))

fig.update_layout(paper_bgcolor = "lavender", font = {'color': "black", 'family': "Arial"})
if not os.path.exists("/home/roycevaughn/Documents/AG_model/images"):
    os.mkdir("/home/roycevaughn/Documents/AG_model/images")
fig.write_image("/home/roycevaughn/Documents/AG_model/images/phosphorus_gauge.png")

# Sulfur Gauge Chart
fig = go.Figure(go.Indicator(
    mode = "gauge+number+delta",
    value = (S_percent_ideal),
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Sulfur", 'font': {'size': 75}},
    delta = {'reference': 100, 'increasing': {'color': "black"}},
    gauge = {
        'axis': {'range': [None, 200], 'tickwidth': 1, 'tickcolor': "black"},
        'bar': {'color': "black"},
        'bgcolor': "black",
        'borderwidth': 2,
        'bordercolor': "black",
        'steps': [
            {'range': [0, 10], 'color': '#00ffd8'},
            {'range': [10, 20], 'color': '#06f7c0'},
            {'range': [20, 30], 'color': '#0beea8'},
            {'range': [30, 40], 'color': '#11e690'},
            {'range': [40, 50], 'color': '#17de78'},
            {'range': [50, 60], 'color': '#1cd560'},
            {'range': [60, 70], 'color': '#22cd48'},
            {'range': [70, 80], 'color': '#28c530'},
            {'range': [80, 90], 'color': '#2dbc18'},
            {'range': [90, 100], 'color': '#33b400'},
            {'range': [100, 110], 'color': '#47a200'},
            {'range': [110, 120], 'color': '#5c9000'},
            {'range': [120, 130], 'color': '#707e00'},
            {'range': [130, 140], 'color': '#856c00'},
            {'range': [140, 150], 'color': '#995a00'},
            {'range': [150, 160], 'color': '#ad4800'},
            {'range': [160, 170], 'color': '#c23600'},
            {'range': [170, 180], 'color': '#d62400'},
            {'range': [180, 190], 'color': '#eb1200'},
            {'range': [190, 200], 'color': '#ff0000'}],
        'threshold': {
            'line': {'color': "navajowhite", 'width': 4},
            'thickness': 0.75,
            'value': 100}}))

fig.update_layout(paper_bgcolor = "lavender", font = {'color': "black", 'family': "Arial"})
if not os.path.exists("/home/roycevaughn/Documents/AG_model/images"):
    os.mkdir("/home/roycevaughn/Documents/AG_model/images")
fig.write_image("/home/roycevaughn/Documents/AG_model/images/sulfur_gauge.png")

In [22]:
# calculating amendment additions
flush = False
# making a list of each amendment and it's constituents
element_name = [ "Nitrogen", "Sulfur", "Phosphorus", "Calcium", "Magnesium", "Potassium", "Sodium", "Hydrogen", "Boron", "Iron", "Manganese", "Copper", "Zinc", "Aluminum", "Chlorine"]
elemental_sulfur = [0, 0.99, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
monopotassium_phosphate = [0.12, 0, 0.61, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
monoammonium_phosphate = [0.12, 0, 0.61, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
calcium_carbonate = [0, 0, 0, 0.3483, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
calcium_sulfate = [0, 0.23553, 0, 0.29439, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
magnesium_sulfate = [0, 0.2664, 0, 0, 0.2019, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
potassium_sulfate = [0, 0.18401, 0, 0, 0, 0.44874, 0, 0, 0, 0, 0, 0, 0, 0, 0]
sodium_chloride = [0, 0, 0, 0, 0, 0, 0.4, 0, 0, 0, 0, 0, 0, 0, 0.6]
sodium_borate = [0, 0, 0, 0, 0, 0, 0.1206, 0, 0.205, 0, 0, 0, 0, 0, 0]
ferrous_sulfate = [0, 0.211, 0, 0, 0, 0, 0, 0, 0, 0.368, 0, 0, 0, 0, 0]
manganese_sulfate = [0, 0.21236, 0, 0, 0, 0, 0, 0, 0, 0, 0.36383, 0, 0, 0, 0]
copper_sulfate = [0, 0.2009, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3981, 0, 0, 0]
zinc_sulfate = [0, 0.19862, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.40497, 0, 0]

#creating a count for ppm added for each amendment
ppm_elemental_sulfur = 0
ppm_monopotassium_phosphate = 0
ppm_monoammonium_phosphate = 0
ppm_calcium_carbonate = 0
ppm_calcium_sulfate = 0
ppm_magnesium_sulfate = 0
ppm_potassium_sulfate = 0
ppm_sodium_chloride = 0
ppm_sodium_borate = 0
ppm_ferrous_sulfate = 0
ppm_manganese_sulfate = 0
ppm_copper_sulfate = 0
ppm_zinc_sulfate = 0

# making a list of each element level, after we amend it (this updates as we add amendments in the model). starting value is where the soil is now.
Nitrogen_amended_ppm = N_ppm
Sulfur_amended_ppm = S_ppm
Phosphorus_amended_ppm = P_ppm
Calcium_amended_ppm = Ca_ppm
Magnesium_amended_ppm = Mg_ppm
Potassium_amended_ppm = K_ppm
Sodium_amended_ppm = Na_ppm
Hydrogen_amended_ppm = H_ppm
Boron_amended_ppm = B_ppm
Iron_amended_ppm = Fe_ppm
Manganese_amended_ppm = Mn_ppm
Copper_amended_ppm = Cu_ppm
Zinc_amended_ppm = Zn_ppm
Aluminum_amended_ppm = Al_ppm
Chlorine_amended_ppm = Cl_ppm

# we put these amended ppm into a list
all_amended_ppms = [Nitrogen_amended_ppm, Sulfur_amended_ppm, Phosphorus_amended_ppm, Calcium_amended_ppm, Magnesium_amended_ppm, Potassium_amended_ppm, Sodium_amended_ppm, Hydrogen_amended_ppm, Boron_amended_ppm, Iron_amended_ppm, Manganese_amended_ppm, Copper_amended_ppm, Zinc_amended_ppm, Aluminum_amended_ppm, Chlorine_amended_ppm]

# make a list of all possible statements that are now just blank spaces
Mg_statement = ""
Ca_statement = "" 
K_statement = "" 
Na_statement = "" 
P_statement = "" 
S_statement = "" 
Fe_statement = "" 
Mn_statement = ""
B_statement = "" 
Cu_statement = "" 
Zn_statement = "" 
Microbe_statement = "" 
S_water_statement = "" 

# finding calcium addiiton. Add calcium carbonate to 60% base saturation, add gypsum up to ideal base sat or until sulfur is maxed, then add cal carb until ideal is reached.
if abs(Calcium_amended_ppm-Ca_ideal_ppm) / Ca_ideal_ppm<0.1:
    Ca_statement = "Calcium is within ideal range."
else:            
    if Ca_percent_ideal < 60:
        calcium_source = calcium_carbonate
        calcium_name = "calcium carbonate"
        ppm_calcium_carbonate = ((TEC_meq*10*(39.76623274 / 2)*(0.6)) - Ca_ppm)/.3483
        Calcium_amended_ppm += ppm_calcium_carbonate/.3483
        calcium_source = "calcium sulfate"
        ppm_calcium_sulfate = ((TEC_meq*10*(39.76623274 / 2)*((Ca_desired_sat/100)-0.6)))/.29439
        Calcium_amended_ppm += ((TEC_meq*10*(39.76623274 / 2)*((Ca_desired_sat/100)-0.6)))
        Ca_statement = "Calcium is low, it's best to bring calcium up by using both high-calcium limestone and gypsum."
        Sulfur_amended_ppm += ppm_calcium_sulfate/0.23553
    if  60 <= Ca_percent_ideal <= Ca_desired_sat and S_ppm < max_S_ppm: # would be better to tie cal sulfate addition to a maximum sulfur appllication.
        calcium_source = calcium_sulfate
        calcium_name = "calcium sulfate"
        ppm_calcium_sulfate = ((TEC_meq*10*(39.76623274 / 2)*((Ca_desired_sat/100)-0.6)))/.29439
        Calcium_amended_ppm += ppm_calcium_sulfate/.29439
        Ca_statement = "Calcium is low - it's best to use gypsum to help correct it."
        Sulfur_amended_ppm += ppm_calcium_sulfate/0.23553
    if  60 <= Ca_percent_ideal <= Ca_desired_sat and S_ppm >= max_S_ppm: 
        calcium_source = calcium_carbonate
        calcium_name = "calcium carbonate"
        ppm_calcium_carbonate = ((TEC_meq*10*(39.76623274 / 2)*(Ca_desired_sat/100)) - Ca_ppm)/.3483
        Calcium_amended_ppm += ppm_calcium_carbonate/.3483
        Ca_statement = "calcium is low, using high-calcium limestone can help fix it."
    if Ca_ppm > Ca_ideal_ppm:
        Ca_statement = "Calcium is high, using nitrogen sources like ammonium sulfate, ammonium nitrate, and sulfur-based amendments can help reduce it over time. It's importart to supply regular irrigation when correcting high calcium levels."


# finding magnesium addition. add magnesium sulfate to achieve base saturation, unless sulfur is initially above maximum allowed. Otherwise, add dolomitic limestone.
if abs(Magnesium_amended_ppm-Mg_ideal_ppm)/Mg_ideal_ppm < 0.10:
    Mg_statement = " Magnesium is within range of ideal."
else:
    if Mg_percent_ideal < 100:
        magnesium_source = magnesium_sulfate
        magnesium_name = "magnesium sulfate"
        ppm_magnesium_sulfate += (Mg_ideal_ppm - Mg_ppm)/.2019
        Magnesium_amended_ppm += (Mg_ideal_ppm - Mg_ppm)
        Sulfur_amended_ppm += (ppm_magnesium_sulfate * 0.17)
        Mg_statement = " Magnesium is low, adding magnesium sulfate can help correct it."
    if Mg_percent_ideal > 100:
        magnesium_source = elemental_sulfur
        magnesium_name = "elemental_surfur"
        ppm_elemental_sulfur += (Mg_ppm - Mg_ideal_ppm)
        Sulfur_amended_ppm += (Mg_ppm - Mg_ideal_ppm)
        Mg_statement = " Magnesium is high, adding elemental sulfur can help correct it."
        flush = True

# finding potassium addition.
if abs(Potassium_amended_ppm - K_ideal_ppm)/K_ideal_ppm <0.10:
    K_statement = " Potassium is within range of ideal."
else:
    if K_ppm < K_ideal_ppm:
        potassium_source = potassium_sulfate
        potassium_name = "potassium_sulfate"
        ppm_potassium_sulfate += (K_ideal_ppm - K_ppm)/0.44874
        Potassium_amended_ppm += (K_ideal_ppm - K_ppm)
        K_statement = " Potassium is low, adding potassium sulfate can help correct it."
    if K_ppm > K_ideal_ppm:
        # this will add 1 ppm sulfur per 1 ppm potassium. That's a very low rate, but it's safer than the full recommendation. It's a similar calculation for eliminating any other cation excess.
        potassium_source = elemental_sulfur
        potassium_name = elemental_sulfur
        ppm_elemental_sulfur += (K_ppm - K_ideal_ppm)/0.44874
        Sulfur_amended_ppm += (K_ppm - K_ideal_ppm)
        K_statement = " Potassium is high, adding elemental sulfur can help correct it."
        flush = True

# finding sodium adition.
if abs(Sodium_amended_ppm - Na_ideal_ppm)/Na_ideal_ppm <0.10:
    Na_statement = " Sodium is within range of ideal."
else:
    if Na_ppm < Na_ideal_ppm: 
        sodium_source = sodium_chloride
        sodium_name = "sodium chloride"
        ppm_sodium_chloride += (Na_ideal_ppm - Na_ppm)/0.4
        Sodium_amended_ppm += (Na_ideal_ppm - Na_ppm)
        Na_statement = " Sodium is low, adding sodium chloride can help correct it."
    if Na_ppm > Na_ideal_ppm:
        Na_statement = " Sodium is high, improving drainage and providing sufficient water to the soil will help flush it out."
    
    if Na_ppm == Na_ideal_ppm:
        Na_statement = " Sodium is within ideal range"

# finding phosphorus addition
if abs(Phosphorus_amended_ppm - P_ideal_ppm)/P_ideal_ppm:
    P_statement = " Phosphorus is within range of ideal."
else:
    if P_ppm < P_ideal_ppm:
        phosphorus_source = monoammonium_phosphate
        phosphorus_name = "monoammonium phosphate"
        ppm_monoammonium_phosphate += (P_ideal_ppm - P_ppm)/0.61
        Phosphorus_amended_ppm += (P_ideal_ppm - P_ppm)
        P_statement = " Phosphorus is low, adding monoammonium phosphate can help correct it."
    if P_ppm > P_ideal_ppm:
        P_statement = " Phosphorus is high, keeping calcium and magnesium at ideal levels can help correct it."
    if P_ppm == P_ideal_ppm:
        P_statement = " Phosphorus is within ideal range."

# finding micronutrient additions
if B_ppm < B_ideal_ppm:
    ppm_sodium_borate += (B_ideal_ppm - B_ppm)/0.205
    Sodium_amended_ppm += (B_ideal_ppm - B_ppm)
    B_statement = " Boron is low, adding sodium borate can help correct it."
if B_ppm > B_ideal_ppm:
    B_statement = " Boron is high, improving drainage and providing sufficient water to the soil will help flush it out."
if B_ppm == B_ideal_ppm:
    B_statement = " Boron is within ideal range."

if Fe_ppm < Fe_ideal_ppm:
    ppm_ferrous_sulfate += (Fe_ideal_ppm - Fe_ppm)/0.205
    Iron_amended_ppm += (Fe_ideal_ppm - Fe_ppm)
    Fe_statement = " Iron is low, adding iron sulfate can help correct it."
if Fe_ppm > Fe_ideal_ppm:
    Fe_statement = " Iron is high, keeping calcium and magnesium at ideal levels can help correct it."
if Fe_ppm == Fe_ideal_ppm:
    Fe_statement = " Iron is within ideal range."

if Mn_ppm < Mn_ideal_ppm:
    ppm_manganese_sulfate += (Mn_ideal_ppm - Mn_ppm)/0.205
    Manganese_amended_ppm += (Mn_ideal_ppm - Mn_ppm)
    Mn_statement = " Manganese is low, adding manganese sulfate can help correct it."
if Mn_ppm > Mn_ideal_ppm:
    Mn_statement = " Manganese is high, improving drainage and providing sufficient water to the soil will help flush it out."
if Mn_ppm == Mn_ideal_ppm:
    Mn_statement = " Manganese is within ideal range."

if Cu_ppm < Cu_ideal_ppm:
    ppm_copper_sulfate += (Cu_ideal_ppm - Cu_ppm)/0.205
    Copper_amended_ppm += (Cu_ideal_ppm - Cu_ppm)
    Cu_statement = " Copper is low, adding copper sulfate can help correct it."
if Cu_ppm > Cu_ideal_ppm:
    Cu_statement = " Copper is high, keeping calcium and magnesium at ideal levels can help correct it."
if Cu_ppm == Cu_ideal_ppm:
    Cu_statement = " Copper is within ideal range."

if Zn_ppm < Zn_ideal_ppm:
    ppm_zinc_sulfate += (Zn_ideal_ppm - Zn_ppm)/0.205
    Copper_amended_ppm += (Zn_ideal_ppm - Zn_ppm)
    Zn_statement = " Zinc is low, adding zinc sulfate can help correct it."
if Zn_ppm > Zn_ideal_ppm:
    Zn_statement = " Zinc is high, keeping calcium and magnesium at ideal levels can help correct it."
if Zn_ppm == Zn_ideal_ppm:
    Zn_statement = " Zinc is within ideal range."

# finding sulfur addition
####if abs():
if Sulfur_amended_ppm < S_ideal_ppm:
    sulfur_source = elemental_sulfur
    sulfur_name = "elemental sulfur"
    ppm_elemental_sulfur += (S_ideal_ppm - S_ppm)
    Sulfur_amended_ppm += (S_ideal_ppm - S_ppm)
    S_statement = " Sulfur is low, adding elemental sulfur can help correct it."
if S_ppm > S_ideal_ppm:
    S_statement = " Sulfur is high, improving drainage and providing sufficient water to the soil will help flush it out."
if S_ppm == S_ideal_ppm:
    S_statement = " Sulfur is within ideal range."

# setting all ppm amendment additions to growing area weight, these are user facing values, in lbs.
calcium_carbonate_user = 0
calcium_sulfate_user = 0
elemental_sulfur_user = 0
monopotassium_phosphate_user = 0
monoammonium_phosphate_user = 0
magnesium_sulfate_user = 0
potassium_sulfate_user = 0
sodium_chloride_user = 0
sodium_borate_user = 0
ferrous_sulfate_user = 0
manganese_sulfate_user = 0
copper_sulfate_user = 0
zinc_sulfate_user = 0

calcium_carbonate_user = float(round((ppm_calcium_carbonate * weight_growing_area)/1000000, 2))
calcium_sulfate_user = float(round((ppm_calcium_sulfate * weight_growing_area)/1000000, 2))
elemental_sulfur_user = float(round((ppm_elemental_sulfur * weight_growing_area)/1000000, 2))
monopotassium_phosphate_user = float(round((ppm_monopotassium_phosphate * weight_growing_area)/1000000, 2))
monoammonium_phosphate_user = float(round((ppm_monoammonium_phosphate * weight_growing_area)/1000000, 2))
magnesium_sulfate_user = float(round((ppm_magnesium_sulfate * weight_growing_area)/1000000, 2))
potassium_sulfate_user = float(round((ppm_potassium_sulfate * weight_growing_area)/1000000, 2))
sodium_chloride_user = float(round((ppm_sodium_chloride * weight_growing_area)/1000000, 2))
sodium_borate_user = float(round((ppm_sodium_borate * weight_growing_area)/1000000, 2))
ferrous_sulfate_user = float(round((ppm_ferrous_sulfate * weight_growing_area)/1000000, 2))
manganese_sulfate_user = float(round((ppm_manganese_sulfate * weight_growing_area)/1000000, 2))
copper_sulfate_user = float(round((ppm_copper_sulfate * weight_growing_area)/1000000, 2))
zinc_sulfate_user = float(round((ppm_zinc_sulfate * weight_growing_area)/1000000, 2))

# setting all ppm amendment additions to growing area weight, user facing values, in grams.
gcalcium_carbonate_user = float(round(((ppm_calcium_carbonate * weight_growing_area)/1000000*453.592), 2))
gcalcium_sulfate_user = float(round(((ppm_calcium_sulfate * weight_growing_area)/1000000*453.592), 2))
gelemental_sulfur_user = float(round(((ppm_elemental_sulfur * weight_growing_area)/1000000*453.592), 2))
gmonopotassium_phosphate_user = float(round(((ppm_monopotassium_phosphate * weight_growing_area)/1000000*453.592), 2))
gmonoammonium_phosphate_user = float(round(((ppm_monoammonium_phosphate * weight_growing_area)/1000000*453.592), 2))
gmagnesium_sulfate_user = float(round(((ppm_magnesium_sulfate * weight_growing_area)/1000000*453.592), 2))
gpotassium_sulfate_user = float(round(((ppm_potassium_sulfate * weight_growing_area)/1000000*453.592), 2))
gsodium_chloride_user = float(round(((ppm_sodium_chloride * weight_growing_area)/1000000*453.592), 2))
gsodium_borate_user = float(round(((ppm_sodium_borate * weight_growing_area)/1000000*453.592), 2))
gferrous_sulfate_user = float(round(((ppm_ferrous_sulfate * weight_growing_area)/1000000*453.592), 2))
gmanganese_sulfate_user = float(round(((ppm_manganese_sulfate * weight_growing_area)/1000000*453.592), 2))
gcopper_sulfate_user = float(round(((ppm_copper_sulfate * weight_growing_area)/1000000*453.592), 2))
gzinc_sulfate_user = float(round(((ppm_zinc_sulfate * weight_growing_area)/1000000*453.592), 2))

#total weight of amendments
sum = gcalcium_carbonate_user + gcalcium_sulfate_user + gelemental_sulfur_user + gmonopotassium_phosphate_user + gmonoammonium_phosphate_user + gmagnesium_sulfate_user + gpotassium_sulfate_user + gsodium_chloride_user + gsodium_borate_user + gferrous_sulfate_user + gmanganese_sulfate_user + gcopper_sulfate_user + gzinc_sulfate_user


# end notes
if ppm_elemental_sulfur > 0:
    S_water_statement = " When using elemental sulfur, microbes will slowly break it down, so it's important to apply when soil temperature is above 50f. Work into the first few inches of soil or spread on the surface."

Microbe_statement = " Soil microbiology is the key to achieving and maintaining ideal soil. It's a good practice to disturb the soil as little as possible, keep the surface covered with mulch or living plants, and provide growing plant roots. These practices will help microbes make nutrients available to plants."        

# generate a paragraph with all the statement variables. ready to insert to pdf.
results_discussion = Ca_statement + Mg_statement + K_statement + Na_statement + P_statement + S_statement + S_water_statement + Fe_statement + Mn_statement + B_statement + Cu_statement + Zn_statement + Microbe_statement

In [23]:
# check the figures
print("SOIL CHARACTERISTICS:", file=output_file)
print("Test type:", test_id, file=output_file)
print("Soil pH:", pH, file=output_file)
print("Soil organic matter percent:", OrganicMatter_percent, file=output_file)
print("Soil type and area:", cultivation_type, "applied to", soil_type, "with primary biological activity depth of approximately", cultivation_depth, "inches.", file=output_file)
print("", file=output_file)

print("POSITIVE CHARGE NUTRIENT SATURATION %", file=output_file)
print("Calcium Ideal Saturation is ", Ca_desired_sat, "% and is currently at ", Ca_sat, "%", sep="", file=output_file)
print("Magnesium Ideal Saturation is ",Mg_desired_sat, "% and is currently at ", Mg_sat, "%",  sep="", file=output_file)
print("Potassium Ideal Saturation is ", K_desired_sat, "% and is currently at ", K_sat, "%",  sep="", file=output_file)
print("Sodium Ideal Saturation is ", Na_desired_sat, "% and is currently at ", Na_sat, "%",  sep="", file=output_file)
print("", file=output_file)

print("KEY RATIOS", file=output_file)
print("Phosphorus to Zinc is: ",P_to_Zn,":1, and ideal is 10:1",  sep="", file=output_file)
print("Calcium to Boron is: ", Ca_to_B, ":1 and ideal is 1000:1",  sep="", file=output_file)
print("Potassium to Iron is: ", K_to_Fe, ":1 and ideal is 4:1",  sep="", file=output_file)
print("Iron to Manganese is: ", Fe_to_Mn, ":1 and ideal is 4:1",  sep="", file=output_file) 
print("Zinc to Copper is: ", Zn_to_Cu, ":1 and ideal is 2:1",  sep="", file=output_file)
print("", file=output_file)

print("NUTRIENT LEVELS", file=output_file)
print("Ca level is: ", Ca_ppm, " ppm. Ideal is: ", Ca_ideal_ppm, ", - ", Ca_percent_ideal, "% of ideal.",  sep="", file=output_file)
print("Mg level is: ", Mg_ppm, " ppm. Ideal is: ", Mg_ideal_ppm, ", - ", Mg_percent_ideal, "% of ideal.",  sep="", file=output_file)
print("K level is: ", K_ppm, " ppm. Ideal is: ", K_ideal_ppm, ", - ", K_percent_ideal, "% of ideal.",  sep="", file=output_file)
print("Na level is: ", Na_ppm, " ppm. Ideal is: ", Na_ideal_ppm, ", - ", Na_percent_ideal, "% of ideal.",  sep="", file=output_file)
print("B level is: ", B_ppm, " ppm. Ideal is: ", B_ideal_ppm, ", - ", B_percent_ideal, "% of ideal.",  sep="", file=output_file)
print("Fe level is: ", Fe_ppm, " ppm. Ideal is: ", Fe_ideal_ppm, ", - ", Fe_percent_ideal, "% of ideal.",  sep="", file=output_file)
print("Mn level is: ", Mn_ppm, " ppm. Ideal is: ", Mn_ideal_ppm, ", - ", Mn_percent_ideal, "% of ideal.",  sep="", file=output_file)
print("Zn level is: ", Zn_ppm, " ppm. Ideal is: ", Zn_ideal_ppm, ", - ", Zn_percent_ideal, "% of ideal.",  sep="", file=output_file)
print("Cu level is: ", Cu_ppm, " ppm. Ideal is: ", Cu_ideal_ppm, ", - ", Cu_percent_ideal, "% of ideal.",  sep="", file=output_file)
print(" ", file=output_file)

print("NUTRIENT ADDITIONS in LB", file=output_file)
print("calcium carbonate:", calcium_carbonate_user, "lb", file=output_file)
print("Calcium sulfate:", calcium_sulfate_user, "lb", file=output_file)
print("Elemental sulfur", elemental_sulfur_user, "lb", file=output_file)
print("Monopotassium phosphate:", monopotassium_phosphate_user, "lb", file=output_file)
print("Monoammonium phosphate:", monoammonium_phosphate_user, "lb", file=output_file) 
print("Magnesium sulfate:", magnesium_sulfate_user, "lb", file=output_file) 
print("Potassium sulfate:", potassium_sulfate_user, "lb", file=output_file)
print("Sodium chloride:", sodium_chloride_user, "lb", file=output_file) 
print("Sodium borate:", sodium_borate_user, "lb", file=output_file) 
print("Ferrous sulfate:", ferrous_sulfate_user, "lb", file=output_file)
print("Manganese sulfate:", manganese_sulfate_user, "lb", file=output_file)
print("Copper sulfate:", copper_sulfate_user, "lb", file=output_file)
print("zinc sulfate:", zinc_sulfate_user, "lb", file=output_file)
print("", file=output_file)

print("NUTRIENT ADDITIONS in Grams", file=output_file)
print("calcium carbonate:", gcalcium_carbonate_user, "grams", file=output_file)
print("Calcium sulfate:", gcalcium_sulfate_user, "grams", file=output_file)
print("Elemental sulfur", gelemental_sulfur_user, "grams", file=output_file)
print("Monopotassium phosphate:", gmonopotassium_phosphate_user, "grams", file=output_file)
print("Monoammonium phosphate:", gmonoammonium_phosphate_user, "grams", file=output_file) 
print("Magnesium sulfate:", gmagnesium_sulfate_user, "grams", file=output_file) 
print("Potassium sulfate:", gpotassium_sulfate_user, "grams", file=output_file) 
print("Sodium chloride:", gsodium_chloride_user, "grams", file=output_file) 
print("Sodium borate:", gsodium_borate_user, "grams", file=output_file) 
print("Ferrous sulfate:", gferrous_sulfate_user, "grams", file=output_file)
print("Manganese sulfate:", gmanganese_sulfate_user, "grams", file=output_file) 
print("Copper sulfate:", gcopper_sulfate_user, "grams", file=output_file)
print("zinc sulfate:", gzinc_sulfate_user, "grams", file=output_file)

print(sum/453.592, "lb", file=output_file)

print(results_discussion, file=output_file)

output_file.close()

In [24]:
pdf = canvas.Canvas("/home/roycevaughn/Documents/AG_model/images/exeter_report.pdf", pagesize=letter)

# Load the image files
calcium_gauge = "/home/roycevaughn/Documents/AG_model/images/calcium_gauge.png"
magnesium_gauge = "/home/roycevaughn/Documents/AG_model/images/magnesium_gauge.png"
potassium_gauge = "/home/roycevaughn/Documents/AG_model/images/potassium_gauge.png"
sodium_gauge = "/home/roycevaughn/Documents/AG_model/images/sodium_gauge.png"
phosphorus_gauge = "/home/roycevaughn/Documents/AG_model/images/phosphorus_gauge.png"
sulfur_gauge = "/home/roycevaughn/Documents/AG_model/images/sulfur_gauge.png"
micro_bar = "/home/roycevaughn/Documents/AG_model/images/micro_bar.png"

# Define the size of gauge chart images
gwidth = 2 * inch
gheight = 1.44 * inch

# Define the font size and style for the text
h1font_size = 20
h1font_style = "Helvetica-Bold"
h2font_size = 17
h2font_style = "Helvetica-Bold"

# Draw gauge charts on canvas
pdf.drawImage(sodium_gauge, 6.35 * inch, 8.75 * inch, width=gwidth, height=gheight)
pdf.drawImage(potassium_gauge, 4.25 * inch, 8.75 * inch, width=gwidth, height=gheight)
pdf.drawImage(magnesium_gauge, 2.15 * inch, 8.75 * inch, width=gwidth, height=gheight)
pdf.drawImage(calcium_gauge, 0.10 * inch, 8.75 * inch, width=gwidth, height=gheight)

pdf.drawImage(phosphorus_gauge, 0.10 * inch, 6.50 * inch, width=gwidth, height=gheight)
pdf.drawImage(sulfur_gauge, 0.10 * inch, 5.0 * inch, width=gwidth, height=gheight)

pdf.drawImage(micro_bar, 2.20 * inch, 4.60 * inch, width=6 * inch, height=3.5 * inch)


# Draw the text on the canvas
pdf.setFont(h1font_style, h1font_size)
pdf.drawString(2 * inch, 10.8 * inch, "Agger Grand Analysis Results")
pdf.setFont(h2font_style, h2font_size)
pdf.drawString(2 * inch, 10.3 * inch, "Primary Positively Charged Nutrients")
pdf.drawString(0.10 * inch, 8.4 * inch, "Primary Negatively")
pdf.drawString(0.10 * inch, 8.1 * inch, "Charged Nutrients")
pdf.drawString(4.20 * inch, 7.9 * inch, "Micro-Nutrients")

# Define the font size and style for the paragraph
font_size = 12
font_style = "Helvetica"
# Create a style sheet
styles = getSampleStyleSheet()
# Use the "Normal" style to create a paragraph
paragraph = Paragraph(results_discussion, styles["Normal"])
pdf.setFont(font_style, font_size)
paragraph.wrapOn(pdf, 6 * inch, 5 * inch) # Adjust the width and height based on your requirements
paragraph.drawOn(pdf, 0.5 * inch, 2 * inch) # Adjust the x and y coordinates based on your requirements

# Save the PDF document
pdf.save()
